# Simulation using the `pulser.sampler` module

This notebook is a work document to investigate how to use the sampler correctly in the simulation module.

In [1]:
import numpy as np

import pulser
import pulser_simulation
from pulser.channels import Rydberg
from pulser.devices import MockDevice
from pulser import Pulse


In [2]:
reg = pulser.Register.square(1)
seq = pulser.Sequence(reg, MockDevice)

seq.declare_channel("main", "rydberg_global")

seq.add(Pulse.ConstantPulse(500, 10, 20, 0), "main")


In [3]:
sim = pulser_simulation.Simulation(seq)
res = sim.run()
print(len(sim.samples["Global"]["ground-rydberg"]["amp"]))
fs1 = res.get_final_state()
print(fs1)

Using legacy sampling
501
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.17230094+0.22978882j]
 [0.95786715+0.j        ]]


In [4]:
config2 = pulser_simulation.SimConfig(use_sampler=True)
sim2 = pulser_simulation.Simulation(seq, config=config2)
res2 = sim2.run()
print(len(sim.samples["Global"]["ground-rydberg"]["amp"]))
fs2 = res2.get_final_state()
print(fs2)

Using the sampler
501
Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.17041513+0.2293127j]
 [0.95831851+0.j       ]
 [0.        +0.j       ]]


The two final state differ: using the sampler, we have a 3-level state.

In [5]:
print(fs1 - fs2)


TypeError: Incompatible quantum object dimensions